In [ ]:
# ==============================
# ✅ 要約スクリプト（ローカル実行版）
# ==============================
import os
from transformers import pipeline
from huggingface_hub import login
login(token="ここにトークンを入れる")

# モデルID（ローカルでダウンロード）
MODEL_ID = "google/gemma-2-2b-it"

def summarize_local(text: str, max_tokens: int = 280, temperature: float = 0.3, char_limit: int = 100):
    """
    Gemma-2-2b-it をローカルで使って日本語の要約を生成します。
    char_limit: 要約の文字数上限
    """
    print("モデルをロード中...（初回は数分かかります）")

    generator = pipeline(
        "text-generation",
        model=MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

    prompt = (
        f"あなたは日本語の要約アシスタントです。以下の文章を{char_limit}文字で自然に要約してください。\n"
        "プレゼントキャンペーンなどの不要な情報は省き、読みやすくまとめてください。\n"
        "要約は1つだけ生成し、繰り返さないでください。\n\n"
        f"【入力】\n{text}\n\n【要約】"
    )

    result = generator(
        prompt,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=generator.tokenizer.eos_token_id
    )

    generated_text = result[0]['generated_text']
    summary = generated_text.replace(prompt, "").strip()

    # 行の重複削除
    lines = summary.split('\n')
    unique_lines = []
    for line in lines:
        if line.strip() and line not in unique_lines:
            unique_lines.append(line.strip())
    summary = '\n'.join(unique_lines[:3])

    # 文字数制限（オーバーしたら切る）
    if len(summary) > char_limit:
        summary = summary[:char_limit] + "..."

    return summary, len(summary)  # ← ★ 文字数も返す！


# ==============================
# ✅ 入力 & 実行
# ==============================
print("📝 要約したいテキストを入力してください（複数行OK）\n")
text = ""
while True:
    line = input()
    if line.strip() == "":
        break
    text += line + "\n"

# 文字数指定
try:
    char_limit = int(input("要約の文字数を指定してください（例: 100）: "))
except ValueError:
    char_limit = 100
    print("無効な入力なので、デフォルトの100文字に設定します。")

print("\n--- 要約結果 ---\n")
summary, length = summarize_local(text, char_limit=char_limit)

print(summary)
print(f"\n📏 要約の文字数: {length} 文字")
